In [ ]:
import os
import pandas as pd
from pyDataverse.api import NativeApi, DataAccessApi

# CONFIGURACIÓN 
API_KEY = "7386f7e6-18cc-49ee-a8eb-d08ac888783d"
BASE_URL = "https://datospararesiliencia.cl"

DATA_DIR = "datos_eventos"
os.makedirs(DATA_DIR, exist_ok=True)

DOIS = [
    "doi:10.71578/FMGXND",
    "doi:10.71578/UXAUN5",
    "doi:10.71578/QRDY49",
    "doi:10.71578/HKLGZ8",
    "doi:10.71578/TLSNSV",
]

VALID_EXT = {".csv", ".xlsx", ".xls"}


api = NativeApi(BASE_URL, API_KEY)
data_api = DataAccessApi(BASE_URL, API_KEY)

# UTILIDADES
# Corregir caracteres maldecodificados 
def fix_encoding(col):
    try:
        return col.encode("latin1").decode("utf8")
    except:
        return col


# Detectar columna Región
def detectar_columna_region(df):
    for col in df.columns:
        c = col.lower().strip()
        if "regi" in c:
            return col
    return None


# Detectar columna Fecha
def detectar_columna_fecha(df):
    posibles = ["fecha", "fecha_update"]
    for col in df.columns:
        if col.lower().strip() in posibles:
            return col
    return None


# Normalizar nombre de región final
def normalizar_region(valor):
    v = str(valor).lower()
    if "bio" in v:
        return "BioBio"
    if "nuble" in v:
        return "Ñuble"
    return None



# API descargar archivos
def download_dataset_files(doi):
    ds = api.get_dataset(doi)

    if ds.status_code != 200:
        return []

    files = ds.json()["data"]["latestVersion"]["files"]
    downloaded_paths = []

    for f in files:
        fname = f["dataFile"]["filename"]
        fid = f["dataFile"]["id"]
        ext = os.path.splitext(fname)[1].lower()

        #print(f" Archivo: {fname}, id={fid}")

        if ext not in VALID_EXT:
            continue

        resp = data_api.get_datafile(fid, is_pid=False)
        if resp.status_code != 200:
            continue

        path = os.path.join(DATA_DIR, fname)
        with open(path, "wb") as f:
            f.write(resp.content)

        downloaded_paths.append(path)

    return downloaded_paths

# PROCESAR Y LIMPIAR TABLAS
def load_and_clean_file(path):
    try:
        # Cargar archivo
        if path.endswith(".csv"):
            df = pd.read_csv(path, encoding="latin-1")
        elif path.endswith(".xlsx") or path.endswith(".xls"):
            df = pd.read_excel(path, engine="openpyxl")
        else:
            return None

        # Corregir encoding de columnas
        df.columns = [fix_encoding(c) for c in df.columns]

        # Separar columnas fusionadas con "|"
        cols_to_split = [c for c in df.columns if "|" in c and df[c].notna().sum() > 0]
        split_frames = []

        for c in cols_to_split:
            new_cols = c.split("|")
            expanded = df[c].astype(str).str.split("|", expand=True)

            if expanded.shape[1] == len(new_cols):
                expanded.columns = new_cols
                split_frames.append(expanded)
                df = df.drop(columns=[c])

        for f in split_frames:
            df = pd.concat([df, f], axis=1)

        # Detectar columnas útiles
        col_region = detectar_columna_region(df)
        if col_region is None:
            return None

        col_fecha = detectar_columna_fecha(df)
        if col_fecha is None:
            return None

        # Normalizar región
        df[col_region] = (
            df[col_region]
            .astype(str)
            .apply(fix_encoding)      # corregir caracteres maldecodificados
            .str.normalize("NFKD")    # normalizar tildes
            .str.encode("ascii", "ignore")  # convertir ñ a n
            .str.decode("utf8")
            .str.lower()
            .str.strip()
        )

        # Filtrar solo regiones relevantes
        mask = df[col_region].str.contains("bio") | df[col_region].str.contains("nuble")
        df = df[mask]

        if df.empty:
            return None

        # Convertir la fecha a formato YYYY-MM-DD
        df[col_fecha] = pd.to_datetime(df[col_fecha], errors="coerce")

        # Eliminar filas sin fecha válida
        df = df.dropna(subset=[col_fecha])

        if df.empty:
            return None

        df[col_fecha] = df[col_fecha].dt.strftime("%Y-%m-%d")

        # Normalizar nombres finales de región
        df["Region"] = df[col_region].apply(normalizar_region)
        df["Fecha"] = df[col_fecha]

        df = df[["Region", "Fecha"]]

        return df

    except Exception as e:
        return None

# PROCESAR TODOS LOS DATASETS
all_clean_dfs = []

for doi in DOIS:
    paths = download_dataset_files(doi)

    for path in paths:
        df_clean = load_and_clean_file(path)
        if df_clean is not None:
            all_clean_dfs.append(df_clean)

if not all_clean_dfs:
    raise RuntimeError("No se pudo cargar ningún archivo válido con región y fecha.")

# UNIFICAR

df_final = pd.concat(all_clean_dfs, ignore_index=True)

output_path = os.path.join(DATA_DIR, "eventos_unificados.csv")
df_final.to_csv(output_path, index=False)

print("Dimensiones finales:", df_final.shape)
print(df_final.head())
print(f"\nGuardado en: {output_path}")


In [ ]:
import pandas as pd
import unicodedata

def fix_encoding(text):
    if not isinstance(text, str):
        return text
    try:
        return text.encode("latin1").decode("utf8")
    except:
        return text

# Cargar archivo
input_file = "inversion_semestres.xlsx"
df = pd.read_excel(input_file)

# Intentar detectar columna Región
def normalize_for_match(s):
    #Normaliza texto eliminando tildes y bajando a minúsculas.
    s = str(s)
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))
    return s.lower().strip()

normalized_cols = {normalize_for_match(c): c for c in df.columns}

possible_region_keys = [
    "region",
    "región",
    "region administrativa",
    "nombre region",
    "nombre región",
]

col_region = None
for key in possible_region_keys:
    key_norm = normalize_for_match(key)
    if key_norm in normalized_cols:
        col_region = normalized_cols[key_norm]
        break

if col_region is None:
    for c in df.columns:
        print(" -", c)
    raise KeyError("No existe columna equivalente a 'Región' en el archivo.")

# Detectar columna Fecha
possible_fecha_keys = ["fecha", "date", "fechas"]

col_fecha = None
for key in possible_fecha_keys:
    key_norm = normalize_for_match(key)
    if key_norm in normalized_cols:
        col_fecha = normalized_cols[key_norm]
        break

if col_fecha is None:
    raise KeyError("No se encontró una columna de fecha.")

# Detectar columna Inversión
possible_inv_keys = [
    "inversion",
    "inversión",
    "inversion (miles de $ de cada año)",
    "monto inversion",
    "monto",
]

col_inv = None
for key in possible_inv_keys:
    key_norm = normalize_for_match(key)
    if key_norm in normalized_cols:
        col_inv = normalized_cols[key_norm]
        break

if col_inv is None:
    for c in df.columns:
        print(" -", c)
    raise KeyError("No existe columna de inversión en el archivo.")

# Normalizar Región
df[col_region] = (
    df[col_region]
    .astype(str)
    .apply(fix_encoding)
    .str.normalize("NFKD")
    .str.encode("ascii", "ignore")
    .str.decode("utf8")
    .str.lower()
    .str.strip()
)

# Variantes aceptadas
bio_vars = ["bio bio", "biobio", "bio-bio"]
nuble_vars = ["nuble", "ñuble"]

df_filtered = df[df[col_region].isin(bio_vars + nuble_vars)].copy()

# Mapear al nombre final
def map_region(r):
    if r in bio_vars:
        return "BioBio"
    if r in nuble_vars:
        return "Ñuble"
    return None

df_filtered["Region"] = df_filtered[col_region].apply(map_region)

# Normalizar fecha
df_filtered["Fecha"] = pd.to_datetime(df_filtered[col_fecha], dayfirst=True, errors="coerce").dt.date

# Normalizar inversión
df_filtered["Inversion"] = (
    df_filtered[col_inv]
    .astype(str)
    .str.replace(r"[^0-9,.\-]", "", regex=True)
    .str.replace(",", ".", regex=False)
    .astype(float)
    .mul(1000)
    .round(0)        # redondea al entero más cercano
    .astype(int)     # convierte a entero
)

# Exportar CSV
output_file = "inversion_filtrada.csv"
df_filtered[["Region", "Fecha", "Inversion"]].to_csv(output_file, index=False)

print("\nArchivo generado correctamente:", output_file)


In [ ]:
"""
ARIMAX forecasting (SARIMAX) para regiones BioBio y Ñuble.
Entrada:
 - eventos_unificados.csv  (columns: Region, Fecha)  Fecha puede ser yyyy-mm-dd o similar
 - inversion_filtrada.csv  (columns: Region, Fecha, Inversion)  Fecha = YYYY-01-01 o YYYY-07-01 (semestres)
Salida:
 - Prints métricas y resumen por región
 - Guarda gráficos: forecast_BioBio.png y forecast_Ñuble.png (si hay datos)
 - Guarda CSVs de forecast: forecast_BioBio.csv, forecast_Ñuble.csv
Notas:
 - Forecast: 10 años = 20 semestres
 - División train/test: 80% train (temporal)
 - Selección de orden (p,d,q) por búsqueda de AIC con p,q ∈ {0,1,2}, d fijo = 1 (puedes cambiar)
"""
import os
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from datetime import datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Configuración 

EVENTS_FILE = "datos_eventos/eventos_unificados.csv"
INV_FILE = "inversion_filtrada.csv"
REGIONS = ["BioBio", "Ñuble"]  # regiones a procesar
FORECAST_YEARS = 10
SEMS_PER_YEAR = 2
FORECAST_PERIODS = FORECAST_YEARS * SEMS_PER_YEAR  # 20 semestres
TRAIN_FRAC = 0.8
MAX_P = 2
MAX_Q = 2
D = 1  # grado de differencing

# Utilidades
def normalize_region_name(s):
    if pd.isna(s):
        return s
    s = str(s).strip()
    low = s.lower()
    if "bio" in low:
        return "BioBio"
    if "ñuble" in low or "nuble" in low:
        return "Ñuble"
    return s

def to_semester_start(ts):
    ts = pd.to_datetime(ts, errors='coerce')
    if pd.isna(ts):
        return pd.NaT
    if ts.month <= 6:
        return pd.Timestamp(year=ts.year, month=1, day=1)
    else:
        return pd.Timestamp(year=ts.year, month=7, day=1)

def load_data():
    if not os.path.exists(EVENTS_FILE):
        raise FileNotFoundError(f"No encuentro {EVENTS_FILE} en el directorio actual.")
    if not os.path.exists(INV_FILE):
        raise FileNotFoundError(f"No encuentro {INV_FILE} en el directorio actual.")
    eventos = pd.read_csv(EVENTS_FILE)
    inversion = pd.read_csv(INV_FILE)
    return eventos, inversion

def prepare(eventos, inversion):
    # Normalizar nombres
    eventos['Region'] = eventos['Region'].apply(normalize_region_name)
    inversion['Region'] = inversion['Region'].apply(normalize_region_name)
    # Parse fecha
    eventos['Fecha'] = pd.to_datetime(eventos['Fecha'], errors='coerce')
    inversion['Fecha'] = pd.to_datetime(inversion['Fecha'], errors='coerce')
    # Mapear eventos a semestre (semester start)
    eventos['Semestre'] = eventos['Fecha'].apply(to_semester_start)
    # Contar eventos por region y semestre
    ev_counts = (eventos
                 .dropna(subset=['Semestre'])
                 .groupby(['Region','Semestre'])
                 .size()
                 .reset_index(name='Eventos'))
    ev_counts = ev_counts.rename(columns={'Semestre':'Fecha'})
    # Asegurar que inversion tenga semestres normalizados (redondear al inicio de semestre)
    inversion['Fecha'] = inversion['Fecha'].apply(to_semester_start)
    # Agregar inversion por region+fecha (si hay duplicados)
    inv_agg = inversion.groupby(['Region','Fecha'], as_index=False)['Inversion'].sum()
    # Merge outer para que entren semestres faltantes
    merged = pd.merge(inv_agg, ev_counts, on=['Region','Fecha'], how='outer')
    merged['Inversion'] = merged['Inversion'].fillna(0.0)
    merged['Eventos'] = merged['Eventos'].fillna(0).astype(int)
    merged = merged.sort_values(['Region','Fecha']).reset_index(drop=True)
    return merged

def ensure_semester_index(df_region):
    # df_region: tiene columnas Fecha, Inversion, Eventos para una region
    df_region = df_region.set_index('Fecha').sort_index()
    # crear rango completo desde primer a ultimo semestre con freq '6MS' anclado en inicio de semestre
    start = df_region.index.min()
    end = df_region.index.max()
    if pd.isna(start) or pd.isna(end):
        return pd.DataFrame(columns=['Inversion','Eventos'])
    idx = pd.date_range(start=start, end=end, freq='6MS')  # cada semestre
    s = df_region.reindex(idx)
    s.index.name = 'Fecha'
    s['Inversion'] = s['Inversion'].fillna(0.0)
    s['Eventos'] = s['Eventos'].fillna(0).astype(int)
    return s

def select_order_by_aic(endog, exog, seasonal_order, max_p=2, max_q=2, d=1):
    best_aic = np.inf
    best_order = (1, d, 1) # Default fallback
    
    for p in range(0, max_p + 1):
        for q in range(0, max_q + 1):
            try:
                # Probamos el modelo COMPLETO (seasonal + non-seasonal)
                # Si hay conflicto de lags (ej: q=2 con s=2), SARIMAX lanzará error aquí
                # y el except lo capturará, saltando esa combinación inválida.
                model = SARIMAX(endog, exog=exog, 
                                order=(p, d, q),
                                seasonal_order=seasonal_order,
                                enforce_stationarity=False, 
                                enforce_invertibility=False)
                
                res = model.fit(disp=False, maxiter=200)
                
                if res.aic < best_aic:
                    best_aic = res.aic
                    best_order = (p, d, q)
            except Exception:
                continue
                
    return best_order

def mape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    denom = np.where(np.abs(y_true) < 1e-9, 1.0, np.abs(y_true))
    return np.mean(np.abs((y_true - y_pred) / denom)) * 100

# Pipeline por región
def run_for_region(merged, region, forecast_periods=FORECAST_PERIODS):
    df_reg = merged[merged['Region'] == region][['Fecha','Inversion','Eventos']].copy()
    if df_reg.empty:
        return None
    df_reg['Fecha'] = pd.to_datetime(df_reg['Fecha'])
    series = ensure_semester_index(df_reg)
    if series.empty:
        return None

    # División train/test temporal (80% train)
    n = len(series)
    train_n = int(np.floor(n * TRAIN_FRAC))
    if train_n < 4:
        train_n = max(1, n - 2)
    train_idx = series.index[:train_n]
    test_idx = series.index[train_n:]

    endog = series['Inversion']
    exog = series[['Eventos']]

    # Define estacionalidad semestral
    # (P=1, D=0, Q=1, s=2)
    s_order = (1, 0, 1, 2)

    # Selección de orden por AIC
    try:
        # s_order pasa a la función 
        order = select_order_by_aic(endog.loc[train_idx], exog.loc[train_idx], 
                                    seasonal_order=s_order, 
                                    max_p=MAX_P, max_q=MAX_Q, d=D)
    except Exception as e:
        order = (1, D, 1)

    # Ajuste modelo 
    model = SARIMAX(endog.loc[train_idx], 
                    exog=exog.loc[train_idx], 
                    order=order,
                    seasonal_order=s_order, 
                    enforce_stationarity=False, 
                    enforce_invertibility=False)
    
    res = model.fit(disp=False, maxiter=200)

    metrics = {}
    if len(test_idx) > 0:
        pred_test_res = res.get_prediction(start=test_idx[0], end=test_idx[-1], exog=exog.loc[test_idx], dynamic=False)
        pred_test = pred_test_res.predicted_mean
        true_test = endog.loc[test_idx]

        # MSE 
        mse = mean_squared_error(true_test, pred_test)

        # RMSE 
        rmse = np.sqrt(mse)

        # MRE 
        try:
            mre = np.mean(np.abs((true_test - pred_test) / true_test))
        except:
            mre = None

        metrics = {
            'MSE': mse, 
            'RMSE': rmse, 
            'MRE': mre,
            'MAPE (%)': mre * 100 if mre is not None else None 
        }
    else:
        pred_test = pd.Series(dtype=float)
        metrics = {'MSE': None, 'RMSE': None, 'MRE': None}

    # Forecast futuro
    last_cycle = exog.iloc[-4:]['Eventos'].values 
    num_cycles = int(np.ceil(forecast_periods / len(last_cycle)))
    future_exog_vals = np.tile(last_cycle, num_cycles)[:forecast_periods]
    
    start_future = series.index[-1] + pd.DateOffset(months=6)
    future_index = pd.date_range(start=start_future, periods=forecast_periods, freq='6MS')

    forecast_res = res.get_forecast(steps=forecast_periods, exog=future_exog_vals.reshape(-1,1))
    forecast_mean = forecast_res.predicted_mean
    forecast_ci = forecast_res.conf_int()

    df_forecast = pd.DataFrame({
        'Fecha': future_index,
        'Forecast': forecast_mean.values,
        'Lower_CI': forecast_ci.iloc[:,0].values,
        'Upper_CI': forecast_ci.iloc[:,1].values,
        'Exog_Pred_Eventos': future_exog_vals
    }).set_index('Fecha')

    out = {
        'region': region,
        'order': order,
        'model_result': res,
        'train_index': train_idx,
        'test_index': test_idx,
        'pred_test': pred_test,
        'true_test': endog.loc[test_idx],
        'metrics': metrics,
        'forecast_df': df_forecast,
        'series': series
    }
    return out

# Main
def main():
    try:
        eventos, inversion = load_data()
    except Exception as e:
        return

    merged = prepare(eventos, inversion)
    # Asegurar que Fecha sea datetime
    merged['Fecha'] = pd.to_datetime(merged['Fecha'])
    # Procesar cada región
    results = {}
    for region in REGIONS:
        res = run_for_region(merged, region)
        if res is None:
            continue
        results[region] = res

        # Imprimir métricas
        print(f"\n=== Resultados para {region} ===")
        print("Order (p,d,q):", res['order'])
        print("Métricas (test):", res['metrics'])
        # resumen del modelo (primeras tablas)
        try:
            print(res['model_result'].summary().tables[0])
        except Exception:
            pass

        # Guardar forecast a CSV
        fname = f"forecast_{region}.csv"
        res['forecast_df'].to_csv(fname, index=True)
        print(f"Forecast guardado en {fname}")

        # Graficar observed, test pred, forecast
        plt.figure(figsize=(12,6))
        series = res['series']
        plt.plot(series.index, series['Inversion'], label='Observed (Inversion)', marker='o')
        if len(res['test_index'])>0 and not res['pred_test'].empty:
            plt.plot(res['pred_test'].index, res['pred_test'].values, label='Predicted (test)', linestyle='--', marker='x')
        plt.plot(res['forecast_df'].index, res['forecast_df']['Forecast'], label=f'Forecast ({FORECAST_YEARS} yrs)', marker='s')
        plt.fill_between(res['forecast_df'].index, res['forecast_df']['Lower_CI'], res['forecast_df']['Upper_CI'], alpha=0.25)
        plt.title(f"ARIMAX Forecast - {region}")
        plt.xlabel("Fecha (semestre)")
        plt.ylabel("Inversión")
        plt.legend()
        plt.grid(True)
        imgname = f"forecast_{region}.png"
        plt.tight_layout()
        plt.savefig(imgname)
        plt.close()

if __name__ == "__main__":
    main()
